In [1]:
import o
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from pylab import rcParams
rcParams['figure.figsize'] = 10, 6
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd
import yfinance as yf
from backtesting import Strategy, Backtest
from backtesting.lib import crossover

Loading BokehJS ...

In [16]:
#input real time plus predicted data
data = yf.download(tickers='BTC-USD', start='2019-01-01')
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-12-31,3866.839111,3868.742920,3725.867432,3742.700439,3742.700439,4661840806
2019-01-01,3746.713379,3850.913818,3707.231201,3843.520020,3843.520020,4324200990
2019-01-02,3849.216309,3947.981201,3817.409424,3943.409424,3943.409424,5244856836
2019-01-03,3931.048584,3935.685059,3826.222900,3836.741211,3836.741211,4530215219
2019-01-04,3832.040039,3865.934570,3783.853760,3857.717529,3857.717529,4847965467
...,...,...,...,...,...,...
2022-02-17,43937.070312,44132.972656,40249.371094,40538.011719,40538.011719,26246662813
2022-02-18,40552.132812,40929.152344,39637.617188,40030.976562,40030.976562,23310007704
2022-02-19,40026.023438,40418.878906,39713.058594,40122.156250,40122.156250,13736557863


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1149 entries, 2018-12-31 to 2022-02-22
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1149 non-null   float64
 1   High       1149 non-null   float64
 2   Low        1149 non-null   float64
 3   Close      1149 non-null   float64
 4   Adj Close  1149 non-null   float64
 5   Volume     1149 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 62.8 KB


In [18]:
def MovingAverage(price, period):
    return pd.Series(price).rolling(period).mean()
def SMA(values, n):
    return pd.Series(values).rolling(n).mean()
class SMACross(Strategy):
    # Define the two MA lags as *class variables*
    ma_short = 10
    ma_long = 20
    df = data
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.df.Close.to_numpy(), self.ma_short)
        self.sma2 = self.I(SMA, self.df.Close.to_numpy(), self.ma_long)
    
    def next(self):
        # If sma1 crosses above sma2 buy the asset
        if crossover(self.sma1, self.sma2):
            self.buy()
# Else, if sma1 crosses below sma2 sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
bt = Backtest(data=data, strategy=SMACross, cash=10000, commission=0)
# evaluate all possible combinations
stats, heatmap = bt.optimize(
    ma_short=range(10, 110, 5),
    ma_long=range(20, 210, 5),
    constraint=lambda p: p.ma_short < p.ma_long,
    maximize='Equity Final [$]',
    return_heatmap=True)


In [19]:
ma_short = stats['_strategy'].ma_short
ma_long = stats['_strategy'].ma_long

In [20]:
data['slow_sma'] = data['Close'].rolling(ma_short).mean()
data['fast_sma'] = data['Close'].rolling(ma_long).mean()
data['prev_fast_sma'] = data['fast_sma'].shift(1)
df = data.copy()
df = df.dropna()
def find_crossover(fast_sma, prev_fast_sma, slow_sma):
    if fast_sma > slow_sma and prev_fast_sma < slow_sma:
        return "bullish crossover"
    if fast_sma < slow_sma and prev_fast_sma > slow_sma:
        return "bearish crossover"
    return "None"

df['crossover'] = np.vectorize(find_crossover)(df['fast_sma'], df['prev_fast_sma'], df['slow_sma'])
signal = df[df['crossover'] == 'bearish crossover'].copy()
signal

,Open,High,Low,Close,Adj Close,Volume,slow_sma,fast_sma,prev_fast_sma,crossover
Date,,,,,,,,,,
2019-11-08,9265.368164,9272.759766,8775.534180,8804.880859,8804.880859,24333037836,8678.913005,8658.057197,8687.287881,bearish crossover
2020-04-22,6879.440430,7145.865723,6867.781738,7117.207520,7117.207520,33249153866,6835.882275,6804.464580,6837.876152,bearish crossover
2020-07-03,9124.842773,9202.344727,9058.794922,9087.303711,9087.303711,13078970999,9425.402181,9415.674941,9428.603301,bearish crossover
2021-05-02,57825.863281,57902.593750,56141.906250,56631.078125,56631.078125,38177405335,57072.686979,56999.669844,57091.910000,bearish crossover
2021-08-03,39178.402344,39750.031250,37782.050781,38152.980469,38152.980469,26189830450,34923.283854,34908.334531,34949.644453,bearish crossover


In [21]:
# print the last 5 days
def sell_buy_hold(df):
    for i in range(5):
        if df.iloc[-i].crossover == 'None':
            print("for ", df.index[-i], ": Hold")
        elif df.iloc[-i].crossover == 'bullish crossover':
            print("for ", df.index[-i], ": Buy")
        else:
            print("for ", df.index[-i], ": Sell")

In [22]:
print(stats)

Start                     2018-12-31 00:00:00
End                       2022-02-22 00:00:00
Duration                   1149 days 00:00:00
Exposure Time [%]                     59.0949
Equity Final [$]                       121573
Equity Peak [$]                        141046
Return [%]                            1115.73
Buy & Hold Return [%]                 894.394
Return (Ann.) [%]                     121.115
Volatility (Ann.) [%]                 112.513
Sharpe Ratio                          1.07645
Sortino Ratio                           4.102
Calmar Ratio                          4.71225
Max. Drawdown [%]                    -25.7021
Avg. Drawdown [%]                    -6.22271
Max. Drawdown Duration      315 days 00:00:00
Avg. Drawdown Duration       25 days 00:00:00
# Trades                                    9
Win Rate [%]                          66.6667
Best Trade [%]                        366.037
Worst Trade [%]                      -11.8669
Avg. Trade [%]                    